## CCTV Live Streams

### These are some public cctv live streams of some countries

<br>

Canada(Toranto) - http://72.138.175.115/mjpg/video.mjpg?camera=1

Canada(Sault Ste. Marie) - http://142.112.106.49:84/mjpg/video.mjpg

India(Bengaluru) - http://117.215.128.17:82/nphMotionJpeg?Resolution=640x480&Quality=Motion

Germany(Spiekeroog) - http://5.61.162.124/mjpg/video.mjpg?camera=1

France(Vorey) - http://88.166.245.77:81/mjpg/video.mjpg

France(Saint-Denis) - http://77.135.210.108/mjpg/video.mjpg

USA(Dallas) - http://98.172.44.107:8090/mjpg/video.mjpg

USA(Dallas) - http://166.251.105.190/mjpg/video.mjpg

USA(Savannah) - http://73.182.17.160:8081/mjpg/1/video.mjpg

Japan(Tokyo) - http://118.22.23.185/nphMotionJpeg?Resolution=640x480&Quality=Standard

Japan(Tokyo) - http://219.101.88.67:60001/nphMotionJpeg?Resolution=640x480&Quality=Clarity

## Install Necessary Modules

In [ ]:
!pip install ultralytics opencv-python
!pip install colab-xterm
%load_ext colabxterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Modules

In [ ]:
from ultralytics import YOLO
import cv2, time, os, csv
import datetime
from dateutil.tz import gettz
from PIL import Image as im
from moviepy.editor import *
from google.colab.patches import cv2_imshow

In [ ]:
# %cd /content

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="T5uMjNxNNTTeAUIUq0fl")
# project = rf.workspace("master-dataset-curation").project("cctv-curation-dataset-poc")
# version = project.version(13)
# dataset = version.download("yolov8")


In [ ]:
# %cd /content/CCTV-Curation-Dataset-PoC-13
# model = YOLO('yolov8n.pt')
# results = model.train(data='coco128.yaml', epochs=100, imgsz=640)

In [ ]:
%cd /content
start = time.time()

# Replace the below url with the above url to choose desired stream
cap = cv2.VideoCapture('http://117.215.128.17:82/nphMotionJpeg?Resolution=640x480&Quality=Motion')#('https://player.vimeo.com/progressive_redirect/playback/210741646/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=37077c1af632c7cb144dffed7e04dcc10ede6939be1b57ead875b6b8874b9cfb')#('https://player.vimeo.com/progressive_redirect/playback/328732843/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=bb455a6abdfae56a5cb1f7a8a7e0e309128d832a3b3e15796c6922b7a771c3ac')#('/content/abcd.mp4')

model = YOLO('yolov8n.pt')#('/content/CCTV-Curation-Dataset-PoC-13/runs/detect/train/weights/best.pt')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
size = (width,height)
fps = 10#cap.get(5)


time_before_start = datetime.datetime.now(tz=gettz('Asia/Kolkata'))
time_now = time_before_start
frame_no = 0
add_sec = 0

output = cv2.VideoWriter('/content/drive/MyDrive/output_summary1.avi', fourcc, fps, size, -1)
class_names = model.names
box_csv = open('output_summary.csv','w')
writer = csv.writer(box_csv)
writer.writerow(['Frame No.', 'X1', 'Y1', 'X2', 'Y2', 'Confidence', 'Class'])

try:
  while cap.isOpened():

    # Read each frame from video stream
    ret, frame = cap.read()

    frame_no += 1

    # In case of no stream stop execution
    if not ret:
      break

     # Convert frame to grayscale for video stabilization
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      # If previous frame exists, perform video stabilization
      if prev_gray is not None:
          # Estimate rigid transformation (rotation, translation) between previous and current frames
          transform = cv2.estimateRigidTransform(prev_gray, gray, fullAffine=False)

          # Apply the estimated transformation to the current frame
          if transform is not None:
              frame_stabilized = cv2.warpAffine(frame, transform, (width, height))

              # Update the frame with the stabilized version
              frame = frame_stabilized

      # Save the current frame for future reference
      prev_gray = gray

    # Predicted result of each frame
    results = model.predict(frame, conf=0.5)

    # Result with various bounding boxes for each frame

    for i in results:

      # Check if there is any detection
      if i.boxes.data.tolist():
        # Save the results with predicted class and normalised bounding box
        results[0].save_txt('abc.txt')
        print(cap.get(5))
        # for each of the object get its bounding box values
        for detection in i.boxes.data.tolist():
          x1, y1, x2, y2, conf, cls = detection

          writer.writerow([frame_no ,x1, y1, x2, y2, conf, cls])
          text_size = cv2.getTextSize(str(time_now.time().strftime('%X')), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
          text_x = max(int((x1 + x2)/2) - text_size[0] // 2, 0)
          text_y = max(int((y1 + y2)/2) - text_size[1] // 2, 0)

          # cv2.putText(frame , text, coordinates, fontstyle, scaling_factor, color, thickness, line_style)
          cv2.putText(frame, str(datetime.datetime.now().strftime('%X')), (int((x1+x2)/2), int((y1+y2)/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

          # Create directory for each objects
          if not os.path.exists(class_names[int(cls)]):
            os.mkdir(class_names[int(cls)])

        frame = cv2.resize(frame, size)

        # Save the numpy array as image with their corresponding timestamps
        data = im.fromarray(frame,)
        data.save(f'{class_names[int(cls)]}/{time.time()}.png',)

        # Write the video file with the frame
        output.write(frame)

      else:
        print('No Detection')

    # cv2_imshow(frame)

    # if cv2.waitKey(1) == ord('q'):
    #   break

    if frame_no % int(fps) == 0:
      add_sec += 1
      time_now = time_before_start + datetime.timedelta(0,add_sec)

except InterruptedError:
  pass

finally:
  output.release()
  cap.release()
  # cv2.destroyAllWindows()

print(time.time() - start)

Streaming output truncated to the last 5000 lines.

0: 480x640 (no detections), 237.9ms
Speed: 4.4ms preprocess, 237.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
No Detection

0: 480x640 (no detections), 240.3ms
Speed: 2.2ms preprocess, 240.3ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
No Detection

0: 480x640 (no detections), 245.0ms
Speed: 2.2ms preprocess, 245.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
No Detection

0: 480x640 (no detections), 246.5ms
Speed: 2.8ms preprocess, 246.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
No Detection

0: 480x640 (no detections), 150.2ms
Speed: 3.0ms preprocess, 150.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
No Detection

0: 480x640 (no detections), 152.0ms
Speed: 2.3ms preprocess, 152.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
No Detection

0: 480x640 (no detections), 158.4ms
Speed: 3.0ms preprocess,

KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/box_csv.csv')

cap = cv2.VideoCapture('/content/drive/MyDrive/output1.avi')

fr_cnt = 0

while cap.isOpened():
  ret, frame = cap.read()

  if not ret:
    break

  fr_cnt += 1

  for i in df[df['Frame No.'] == fr_cnt].iterrows():
    cv2.rectangle(frame, (int(i[1].X1), int(i[1].Y1)), (int(i[1].X2), int(i[1].Y2)), (255,0,0))
    cv2_imshow(frame)

cap.release()

In [ ]:
path="/content/drive/MyDrive/output1.avi"

clip=VideoFileClip(path)
clip.ipython_display(width=size[0],maxduration=120)

In [ ]:
import os

def create_video():
  li = os.listdir('car')
  li = sorted(li)
  out = cv2.VideoWriter('/content/drive/MyDrive/output_image.avi', fourcc, 30, size)
  for i in li:
    frame = cv2.imread('car/'+i)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(rgb_frame)
  out.release()

create_video()

In [ ]:
%xterm

# start = time.time()

# # Replace the below url with the above url
# cap = cv2.VideoCapture('http://98.172.44.107:8090/mjpg/video.mjpg')#('https://player.vimeo.com/progressive_redirect/playback/210741646/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=37077c1af632c7cb144dffed7e04dcc10ede6939be1b57ead875b6b8874b9cfb')#('https://player.vimeo.com/progressive_redirect/playback/328732843/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=bb455a6abdfae56a5cb1f7a8a7e0e309128d832a3b3e15796c6922b7a771c3ac')#('/content/abcd.mp4')

# model = YOLO('yolov8n.pt')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
# size = (width,height)
# fps = cap.get(5)


# time_before_start = datetime.datetime.now(tz=gettz('Asia/Kolkata'))
# time_now = time_before_start
# frame_no = 0
# add_sec = 0

# output = cv2.VideoWriter('output.mp4', fourcc, fps, size)
# class_names = model.names

# try:
#   while cap.isOpened():

#     # Read each frame from video stream
#     ret, frame = cap.read()

#     frame_no += 1

#     # In case of no stream stop execution
#     if not ret:
#       break

#     # Predicted result of each frame
#     results = model.predict(frame, conf=0.5)

#     # Result with various bounding boxes for each frame

#     for i in results:

#       # Check if there is any detection
#       if i.boxes.data.tolist():
#         # Save the results with predicted class and normalised bounding box
#         results[0].save_txt('abc.txt')

#         # for each of the object get its bounding box values
#         for detection in i.boxes.data.tolist():
#           x1, y1, x2, y2, conf, cls = detection

#           # Calculate the center coordinates of the bounding box
#           center_x = int((x1 + x2) / 2)
#           center_y = int((y1 + y2) / 2)

#           # Draw the timestamp near the object
#           timestamp_text = str(time_now.time().strftime('%X'))
#           text_size = cv2.getTextSize(timestamp_text, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
#           text_x = max(center_x - text_size[0] // 2, 0)
#           text_y = max(center_y - text_size[1] // 2, 0)
#           cv2.putText(frame, timestamp_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

#           # Draw the bounding box
#           cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

#           # Create directory for each object class
#           class_dir = class_names[int(cls)]
#           if not os.path.exists(class_dir):
#               os.mkdir(class_dir)

#           # Save the frame with the bounding box and timestamp
#           frame_with_box = frame.copy()
#           cv2.putText(frame_with_box, timestamp_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
#           cv2.rectangle(frame_with_box, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
#           cv2.imwrite(f'{class_dir}/{time.time()}.png', frame_with_box)

#       # Write the video file with the frame
#       output.write(frame)
# except InterruptedError:
#   pass

# finally:
#   output.release()
#   cap.release()
#   cv2.destroyAllWindows()

# print(time.time() - start)



In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/avi;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/avi">
</video>
""" % data_url)

This cell output is too large and can only be displayed while logged in.


In [ ]:
import datetime
from dateutil.tz import gettz


a= datetime.datetime.now(tz=gettz('Asia/Kolkata'))
b = a + datetime.timedelta(0,5)

print(a.time().strftime('%X'))
print(b.time().strftime('%X'))

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git


In [ ]:
!cd /content/yolov9/

In [ ]:
!pip install -r yolov9/requirements.txt

In [ ]:
%cd yolov9

In [ ]:
!python val.py --data data/coco.yaml --img 640 --batch 32 --conf 0.001 --iou 0.7 --device cpu --weights './yolov9-c-converted.pt' --save-json --name yolov9_c_c_640_val

In [ ]:
!python detect.py --source './data/images/horses.jpg' --img 640 --device 0 --weights './yolov9-c-converted.pt' --name yolov9_c_c_640_detect

In [ ]:
%cd /content
!unzip CCTV-Curation-Dataset-PoC-13-20240317T184212Z-001 -d /content

In [ ]:
datetime.datetime.now()

In [ ]:
!ffmpeg -i output1.avi output.mp4